In [1]:
import sys
import time
import json
import os

import recommend as r
from train_updated import train
from api.spotifyAPI import SpotifyAPI
from utils import load_graph
from utils import load_features
from sklearn.neighbors import NearestNeighbors
import pandas as pd
from dgl import load_graphs
import matplotlib.pyplot as plt
import networkx as nx
import dgl

import torch
import numpy as np

Using backend: pytorch


# Loading Data

In [2]:
%%time
# Loading Graph ~ 3min
feat_dir = "../../data/a13group1/460k_songset_features.csv"
scratch_pickle_dir = "../../data/a13group1/"
feat_data, uri_map = load_features(feat_dir, scratch_pickle_dir, False, playlist_num=100000)
graph_dir = ("../../data/a13group1/graph_460k.gpickle")
dgl_G, weights = load_graph(graph_dir, uri_map)

Loading feature data...
Feature data shape: torch.Size([461880, 13])
Loading graph data...
Graph Info:
 Name: G
Type: Graph
Number of nodes: 461880
Number of edges: 106486690
Average degree: 461.1011
CPU times: user 8min 13s, sys: 1min 24s, total: 9min 37s
Wall time: 9min 23s


In [3]:
#dgl_G_cuda = dgl_G.to('cuda')
#weights = weights.to('cuda')
#feat_data = feat_data.to('cuda')

# Training Model (optional)

In [4]:
# %%time
# # Training the Model. GPU ~ 00:00:40. CPU ~ 00:53:00.
# with open('../../config/model-params.json') as fh:
#             model_cfg = json.load(fh)
# model, pred, measures = train(dgl_G, weights.to('cpu'), feat_data, cuda=False, feat_dim=13, emb_dim=10, test_data=False)

# # Put everything on CPU
# model = model.to('cpu')
# pred = pred.to('cpu')

# torch.save(model, '460k_1epoch_model.pt')
# torch.save(pred, '460k_1epoch_pred.pt')

# with open("460k_1epoch_measures.json", "w") as out_measures:
#     json.dump(measures, out_measures)

In [5]:
# print('Final Training Classification Report: ')
# print('AUC: 0.8236135679618413')
# print('Final Loss: 0.5051630735397339')
# print('Training Time: ~ 1 Hour (CPU)')
# pd.DataFrame(data={'Measure': [0.0, 1.0, 'accuracy', 'macro avg', 'weighted avg'], 'Precision': [0.49, 0.90, None, 0.69, 0.80], 'Recall': [0.75, 0.74, None, 0.75, 0.75], 'F1-Score': [0.59, 0.82, 0.75, 0.70, 0.76], 'Support': [168110, 518464, 686574, 686574, 686574]})

# Loading Pre-Existing Model

In [6]:
# Load Pre-Existing Model
model = torch.load('460k_model.pt')
pred = torch.load('460k_pred.pt')
model.eval()
pred.eval()

MLPPredictor(
  (W1): Linear(in_features=20, out_features=10, bias=True)
  (W2): Linear(in_features=10, out_features=1, bias=True)
)

# Create Predictions

In [7]:
dgl_G.edata[dgl.EID] = torch.tensor(np.arange(dgl_G.number_of_edges()))

In [ ]:

# Create Embeddings
z = model(dgl_G, feat_data, weights)

# Create Predictions
# In the form of the strength of the connection between source, destination from dgl_G.edges()
preds = pred(dgl_G, z)

In [ ]:
# Create Nearest Neighbors
neigh = NearestNeighbors(n_neighbors=25, radius=0.4)
neigh.fit(feat_data)

In [ ]:
# Get a random playlist
item = r.get_random_playlist()

In [ ]:
item

In [ ]:
# Get playlist information
item, original_tracks, seeds = r.get_playlist_info(item)
original_tracks

In [ ]:
# Get recommendations
uri_recs = r.recommend(seeds, dgl_G, z, pred, neigh, feat_data, uri_map)
uri_recs

# Decode Results with Spotify API

In [ ]:
# Translate to Song Names
client_id = 'ad2536ed7a914d66b89b80fb3a500787'
client_secret = '8c5f45fb008d4bc5bf909ec46d076b65'

spotify = SpotifyAPI(client_id, client_secret)

In [ ]:
%%time
rec_track_names = r.get_rec_names(uri_recs, spotify, 2)

In [ ]:
original_tracks

In [ ]:
rec_track_names.sort(reverse=True, key=lambda x: x[1])

In [ ]:
rec_track_names

Note: We kept repeats because a song that is recommended as the best more than once throughout the playlist should be ranked higher

# Making Batch Recommendations ~ 15 min

In [ ]:
# %%time 
# all_accs = []
# for each in range(200):
#     item = r.get_random_playlist()
#     item, original_tracks, seeds = r.get_playlist_info(item)
#     uri_recs = r.recommend(seeds, dgl_G, z, pred, neigh, feat_data, uri_map)
    
#     checks = []
#     for i in range(len(uri_recs)):
#         well = uri_map[uri_recs[i]] in dgl_G.out_edges(uri_map[seeds[i]])[1]
#         checks.append(well)
        
#     acc = np.count_nonzero(checks) / len(checks)
#     all_accs.append({'pid': item['pid'], 'seeds': seeds, 'recs':uri_recs, 'accuracy': acc})
    
# with open("../analysis/batch_recommendations.json", "w") as final:
#     json.dump(all_accs, final)

In [ ]:
# with open("../analysis/uri_map_170k.json", "w") as final:
#     json.dump([uri_map], final)